<a href="https://colab.research.google.com/github/vijay-lab/internship/blob/master/latest_Smriti_scrapper%20V%202.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

In [0]:
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')

In [0]:
#loading ther chrome driver for colab ## driver = webdriver.Chrome('chromedriver',options=options)

In [0]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jul 18 23:44:43 2019

@author: TAPAN
"""

import selenium
import requests
import numpy as np

from collections import OrderedDict
import pandas as pd
from selenium import webdriver


letters_val=['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','y','z']
# 'a','b','c','d','e',

columns=['movie','song','singers','lyricist','year','music_director','lyrics']
songs_df_main=pd.DataFrame()

# setting to run colab headless. 
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')
driver = webdriver.Chrome("chromedriver",options=options)

from selenium.common.exceptions import NoSuchElementException,ElementClickInterceptedException,ElementNotVisibleException

for letter in letters_val:
    print('currently scraping singers by letter',letter)
    address="http://smriti.com/hindi-songs/singers-"+letter  # selects singer letter 
    driver.get(address)
    #singers=[]
    songs_df_main=pd.DataFrame()
    for singer in range(2,1000):
        print(singer-1,'no singer is being scrapped')
        try: #clicks on singer name
            driver.find_element_by_xpath(f'//*[@id="content"]/div/div/a[{singer}]').click()
            total_songs=[]
            
            total_songs.append(driver.find_elements_by_class_name('onesong'))
            no_of_songs=len(total_songs[0])+1
            
            for elem in range(1,no_of_songs): # this is data downloading block
                com_list=[]
                print('currently scraping song no:',elem, 'out of',no_of_songs-1)
                try:
                    
                    driver.find_element_by_xpath('//*[@id="content"]/div/div/div[{}]/a[2]'.format(elem)).click()
                    
                    movie=driver.find_element_by_xpath('//*[@id="content"]/div/div/ul/li[1]/a').text
                    if not movie:
                        com_list.append(np.nan)
                    else:
                        com_list.append(movie)
                    
                    song=driver.find_element_by_xpath('//*[@id="content"]/div/div/h1').text
                    if not song:
                        com_list.append(np.nan)
                    else:
                        com_list.append(song)
                    singers=driver.find_element_by_xpath('//*[@id="content"]/div/div/ul/li[2]').text
                    if not singers:
                        com_list.append(np.nan)
                    else:
                        com_list.append(singers)
                        
                    lyricist=driver.find_element_by_xpath('//*[@id="content"]/div/div/ul/li[4]').text
                    if not lyricist:
                        com_list.append(np.nan)
                    else:
                        com_list.append(lyricist)
                        
                    year=driver.find_element_by_xpath('//*[@id="content"]/div/div/ul/li[6]/a[1]').text
                    if not year:
                        com_list.append(np.nan)
                    else:
                        com_list.append(year)
                    try:    
                        com_list.append(driver.find_element_by_xpath('//*[@id="content"]/div/div/ul/li[3]/a').text)
                    except NoSuchElementException:
                        com_list.append(np.nan)
                        
                    lyrics=driver.find_element_by_xpath('//*[@id="content"]/div/div/div[1]').text
                    if not lyrics:
                        com_list.append(np.nan)
                    else:
                        com_list.append(lyrics)
                    
                    temp_df = pd.DataFrame(com_list).values.reshape(1,-1) 
                    xx=pd.DataFrame(temp_df,columns=columns)
                    
                    songs_df_main=pd.concat([songs_df_main,xx], ignore_index=True)
                    #songs_df_main.to_csv(f'songs data_{letter}.csv',encoding='utf-8') # Enable this to save songs in csv as they iterate.
                    
                    driver.back()
                except NoSuchElementException:
                    
                    print("oops song doesn't exist.. moving on to next song ")
                    pass
              
                except Exception as e:
                    print(e)
                    pass
                
            driver.back() 
            
        except Exception as e:
          songs_df_main.to_csv(f'songs data_{letter}.csv',encoding='utf-8') # dumps data to csv after finishing fetching of each letter 
          print("oops singer doesn't exist.. moving on to next page ") 
          print(e)
          driver.back()
          pass
        
        finally :
          songs_df_main.to_csv(f'songs data_{letter}.csv',encoding='utf-8') # dumps data to csv after finishing fetching of each letter


print('All done ..phew :)')    